## Data Ingestion from Lakehouse to the Notebook

In [1]:
# importing libraries
%pip install imblearn

# to reduce warnings in results
import warnings
warnings.filterwarnings("ignore")

StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 8, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 15.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [2]:
# Getting dataset from Lakehouse
df = (
    spark.read.option("header", True)
    .option("InferSchema", True)
    .csv("Files/credit_risk_dataset.csv")
    .cache()
)

StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 10, Finished, Available)

In [3]:
# converting from Spark dataframe to Panda dataframe

df = df.toPandas()

StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 11, Finished, Available)

## Data Exploration and Cleaning (with Data Wrangler)

In [4]:
# importing libraries needed Pandas, Matplotlib, Numpy, Itertools, and Seaborn

import seaborn as sns
sns.set_theme(style="whitegrid", palette="tab10", rc = {'figure.figsize':(9,6)})
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import rc, rcParams
import numpy as np
import pandas as pd
import itertools


StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 12, Finished, Available)

In [5]:
# previewing the dataset
display(df, summary=True)

StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, b1445e94-2287-44a4-b959-5523dceaa964)

In [6]:
# previewing the top 5 rows
df.head(5)

StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 14, Finished, Available)

,person_home_ownership,loan_intent,loan_grade,cb_person_default_on_file,person_income,person_age,person_emp_length,loan_amnt,loan_int_rate,cb_person_cred_hist_length,loan_percent_income,loan_status
0,RENT,PERSONAL,D,Y,59000,22,123.0,35000,16.02,3,0.59,1
1,OWN,EDUCATION,B,N,9600,21,5.0,1000,11.14,0,0.10,0
2,MORTGAGE,MEDICAL,C,N,9600,25,1.0,5500,12.87,0,0.57,1
3,RENT,MEDICAL,C,N,65500,23,4.0,35000,15.23,0,0.53,1
4,RENT,MEDICAL,C,Y,54400,24,8.0,35000,14.27,4,0.55,1


In [7]:
# Intial Shape
print('Intial Shape:')
print(f'Rows: {df.shape[0]}\nColumns: {df.shape[1]}\n')

# removing duplicates
df = df.drop_duplicates()

# Printing Data Shape 
print("Shape after removing duplicates")
print(f"Rows: {df.shape[0]}\nColumns: {df.shape[1]}")

StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 15, Finished, Available)

Intial Shape:
Rows: 32581
Columns: 12

Shape after removing duplicates
Rows: 32029
Columns: 12


In [8]:
# Defining the columns dtypes in the dataset
numericals = [col for col in df.columns if df[col].dtypes != "object"]

categoricals = [col for col in df.columns if df[col].dtypes == "object"]

StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 16, Finished, Available)

**Using Data Wrangler to do initial data cleaning and exploration (it can also be done with codes)**

In [9]:
# Code generated by Data Wrangler for pandas DataFrame

def clean_data(df):
    # Replace all instances of "Other" with "RENT" in column: 'person_home_ownership'
    df.loc[df['person_home_ownership'].str.lower() == "Other".lower(), 'person_home_ownership'] = "RENT"

    # Replace all instances of "Y" with "1" in column: 'cb_person_default_on_file'
    df['cb_person_default_on_file'] = df['cb_person_default_on_file'].str.replace("Y", "1", case=False, regex=False)
    
    # Replace all instances of "N" with "0" in column: 'cb_person_default_on_file'
    df['cb_person_default_on_file'] = df['cb_person_default_on_file'].str.replace("N", "0", case=False, regex=False)

    # Replace missing values with the mean of each column in: 'person_emp_length'
    df = df.fillna({'person_emp_length': df['person_emp_length'].mean()})

    # Replace missing values with the mean of each column in: 'loan_int_rate'
    df = df.fillna({'loan_int_rate': df['loan_int_rate'].mean()})

    # Filter rows based on column: 'person_age'
    df = df[df['person_age'] <= 40]

    # Filter rows based on column: 'person_emp_length'
    df = df[df['person_emp_length'] < 26]

    # Filter rows based on column: 'person_income'
    df = df[df['person_income'] <= 27700]
    
    # Filter rows based on column: 'loan_amnt'
    df = df[df['loan_amnt'] <= 12000]

    return df

df_clean = clean_data(df.copy())
df_clean.head()

StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 17, Finished, Available)

,person_home_ownership,loan_intent,loan_grade,cb_person_default_on_file,person_income,person_age,person_emp_length,loan_amnt,loan_int_rate,cb_person_cred_hist_length,loan_percent_income,loan_status
1,OWN,EDUCATION,B,0,9600,21,5.0,1000,11.14,0,0.10,0
2,MORTGAGE,MEDICAL,C,0,9600,25,1.0,5500,12.87,0,0.57,1
5,OWN,VENTURE,A,0,9900,21,2.0,2500,7.14,0,0.25,1
9,OWN,VENTURE,D,0,10000,21,6.0,1600,14.74,0,0.16,1
11,OWN,HOMEIMPROVEMENT,A,0,10000,21,2.0,4500,8.63,0,0.45,1


In [10]:
display(df_clean, summary=True)

StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4d0b2eac-8632-48eb-849a-ee2603ff7247)

****

## Data Pre-Processing

Encoding Task

In [11]:
df_clean.head(5)

StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 19, Finished, Available)

,person_home_ownership,loan_intent,loan_grade,cb_person_default_on_file,person_income,person_age,person_emp_length,loan_amnt,loan_int_rate,cb_person_cred_hist_length,loan_percent_income,loan_status
1,OWN,EDUCATION,B,0,9600,21,5.0,1000,11.14,0,0.10,0
2,MORTGAGE,MEDICAL,C,0,9600,25,1.0,5500,12.87,0,0.57,1
5,OWN,VENTURE,A,0,9900,21,2.0,2500,7.14,0,0.25,1
9,OWN,VENTURE,D,0,10000,21,6.0,1600,14.74,0,0.16,1
11,OWN,HOMEIMPROVEMENT,A,0,10000,21,2.0,4500,8.63,0,0.45,1


In [13]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False, drop='first')  # drop='first' to avoid multicollinearity if needed

# Fit and transform the data
encoded_columns = encoder.fit_transform(df_clean[['person_home_ownership', 'loan_intent']])

# Create a DataFrame with the encoded columns
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['person_home_ownership', 'loan_intent']))

# Concatenate the original DataFrame with the encoded columns
df_encoded = pd.concat([df_clean.drop(['person_home_ownership', 'loan_intent'], axis=1), encoded_df], axis=1)

# Reset the index
df_encoded.reset_index(drop=True, inplace=True)

# Display the encoded DataFrame
print("\nEncoded DataFrame with Reset Index:")
df_encoded.head(5)



StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 21, Finished, Available)


Encoded DataFrame with Reset Index:


,loan_grade,cb_person_default_on_file,person_income,person_age,person_emp_length,loan_amnt,loan_int_rate,cb_person_cred_hist_length,loan_percent_income,loan_status,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE
0,B,0,9600.0,21.0,5.0,1000.0,11.14,0.0,0.10,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,C,0,9600.0,25.0,1.0,5500.0,12.87,0.0,0.57,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,A,0,9900.0,21.0,2.0,2500.0,7.14,0.0,0.25,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,D,0,10000.0,21.0,6.0,1600.0,14.74,0.0,0.16,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,A,0,10000.0,21.0,2.0,4500.0,8.63,0.0,0.45,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [14]:
from sklearn.preprocessing import OrdinalEncoder

# Initialize the OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Fit and transform the data for 'loan_grade'
df_encoded['loan_grade'] = ordinal_encoder.fit_transform(df_encoded[['loan_grade']])


# Display the DataFrame with the ordinal encoded column
print("\nDataFrame with Ordinal Encoded Column:")
df_encoded.head(5)


StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 22, Finished, Available)


DataFrame with Ordinal Encoded Column:


,loan_grade,cb_person_default_on_file,person_income,person_age,person_emp_length,loan_amnt,loan_int_rate,cb_person_cred_hist_length,loan_percent_income,loan_status,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE
0,1.0,0,9600.0,21.0,5.0,1000.0,11.14,0.0,0.10,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2.0,0,9600.0,25.0,1.0,5500.0,12.87,0.0,0.57,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0,9900.0,21.0,2.0,2500.0,7.14,0.0,0.25,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3.0,0,10000.0,21.0,6.0,1600.0,14.74,0.0,0.16,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0,10000.0,21.0,2.0,4500.0,8.63,0.0,0.45,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 24, Finished, Available)

SynapseWidget(Synapse.DataFrame, a9e3db62-680d-49ec-aaf7-927b119fa652)

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# removing NaN Values
df_encoded_clean = df_encoded.dropna()

# Assuming df_encoded is your preprocessed dataset
X = df_encoded_clean.drop('loan_status', axis=1)
y = df_encoded_clean['loan_status']               

# Step 1: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 2: Apply SMOTE to the training set
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Step 3: Standardize the features
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and test data
X_train_res_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)

StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 26, Finished, Available)

2024-06-04:14:11:03,438 WARNING  [tracking_store.py:153] log_inputs not supported


## Model Training
jolly_ice experiment has been created for the Encoding Task

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import logging

# Initialize the logger for Microsoft Fabric
logging.basicConfig(level=logging.INFO)

# Train Logistic Regression model
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train_res_scaled, y_train_res)

# Predict on the test set
y_pred_lr = lr_model.predict(X_test_scaled)

# Calculate performance metrics
conf_matrix_lr = confusion_matrix(y_test, y_pred_lr)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr, average='weighted')
recall_lr = recall_score(y_test, y_pred_lr, average='weighted')
f1_lr = f1_score(y_test, y_pred_lr, average='weighted')

# Log the results in Microsoft Fabric
logging.info(f"Logistic Regression - Confusion Matrix:\n{conf_matrix_lr}")
logging.info(f"Logistic Regression - Accuracy: {accuracy_lr}")
logging.info(f"Logistic Regression - Precision: {precision_lr}")
logging.info(f"Logistic Regression - Recall: {recall_lr}")
logging.info(f"Logistic Regression - F1 Score: {f1_lr}")

StatementMeta(, e01096e4-7024-4fe0-b41e-c444fffef0da, 28, Finished, Available)

2024-06-04:14:21:17,270 WARNING  [tracking_store.py:153] log_inputs not supported
